In [1]:
# %pip install datasets matplotlib torch shap
# %pip install transformers
# %pip install lime

In [ ]:

from custom_model import EmotionClassifier
from datasets import load_dataset 
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader 
import shap 

/home/iwaniukooo/Documents/Projects/wb2-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

with open("dict.txt", "r") as f:
    emotion_dict = [line.strip() for line in f]

emotion_dict

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [ ]:



classifier = EmotionClassifier(device='cpu')  




print("Loading go_emotions dataset...")
dataset = load_dataset("go_emotions", "simplified")





data_subset = dataset['train'].shuffle(seed=42).select(range(4500))  

all_texts = data_subset['text']
all_labels = data_subset['labels'] 
all_labels = [label[0] for label in all_labels]  
all_labels = [emotion_dict[int(label)] for label in all_labels]


train_texts, val_texts = all_texts[:4000], all_texts[4000:4500]
train_labels, val_labels = all_labels[:4000], all_labels[4000:4500]
print(f"Loaded {len(train_texts)} training samples and {len(val_texts)} validation samples.")

Loading go_emotions dataset...
Loaded 4000 training samples and 500 validation samples.


In [ ]:

import random
random_index = random.randint(0, len(train_texts) - 1)
print(f"Sample text: {train_texts[random_index]}")
print(f"Sample labels: {train_labels[random_index]}")

Sample text: This way it becomes undeniably clear that it is the [NAME] who are the problem.
Sample labels: neutral


In [ ]:



print("Starting training...")
classifier.train(
    train_texts, train_labels,
    val_texts, val_labels,
    learning_rate=0.0001,
    epochs=4,  
    batch_size=16  
)

Starting training...
Labels setup: 28 unique labels found: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
Epoch 1/4
  Train Loss: 2.1916
  Val Accuracy: 0.4800
  Val F1 Macro: 0.2748
  Val ROC AUC (Macro OVR): 0.0000
  New best model saved with F1 Macro: 0.2748
Epoch 2/4
  Train Loss: 1.5111
  Val Accuracy: 0.4560
  Val F1 Macro: 0.2713
  Val ROC AUC (Macro OVR): 0.0000
Epoch 3/4
  Train Loss: 0.9935
  Val Accuracy: 0.4720
  Val F1 Macro: 0.3430
  Val ROC AUC (Macro OVR): 0.0000
  New best model saved with F1 Macro: 0.3430
Epoch 4/4
  Train Loss: 0.5955
  Val Accuracy: 0.4320
  Val F1 Macro: 0.3145
  Val ROC AUC (Macro OVR): 0.0000
Training finished. Loading best model state.


In [9]:


#test the model on example text
example_text = "i want to cry, depression is coming"
predicted_labels = classifier.predict([example_text])
print(f"Predicted labels for '{example_text}': {predicted_labels}")

Predicted labels for 'i want to cry, depression is coming': {'sadness': 0.7989416122436523}


In [ ]:
# # 6. Show SHAP explanation (for one example)
# print("Generating SHAP explanation...")
# shap_explanation = classifier.explain_shap(test_text)
# shap.plots.text(shap_explanation)


Generating SHAP explanation...


NameError: name 'test_text' is not defined

In [ ]:

# # 7. Show LIME explanation
# print("\nGenerating LIME explanation...")
# lime_explanation = classifier.explain_lime(test_text)
# lime_explanation.show_in_notebook()
